# Text Data

Version: 2024-10-14

Trying to construct models that understand text falls under the field of *natural language processing*. This is a field of enormous practical importance: chatbot, automated translation and generated new articles area few notable applications. In this notebook we will look into some basic ways of processing text data.

Below is what you might get in a typical dataset of review data:

In [1]:
#Text data
corpus = [
    "This is good.",
    "This is bad.",
    "This is very good.",
    "This is not good.",
    "This is not bad.",
    "This is...is bad."
]

ratings = [
    1,
    0,
    1,
    0,
    1,
    0
]

When analyzing review data the typical goal is to predict a single value, the rating, from the written text. This is a form of *sentiment analysis*. In the case of chatbot and automated translation, where one single value is not sufficient to represent the meaning of text, a vector is outputed by the model instead.

### A. N-gram

Let us count the number of times each word appears in a sample. This is called *unigram* in natural language processing. To do so, we will use ```CountVectorizer``` of scikit-learn:

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
print(X.toarray())

[[0 1 1 0 1 0]
 [1 0 1 0 1 0]
 [0 1 1 0 1 1]
 [0 1 1 1 1 0]
 [1 0 1 1 1 0]
 [1 0 2 0 1 0]]


Use ```get_feature_names_out()``` to see which word each column represents:

In [3]:
vectorizer.get_feature_names_out()

array(['bad', 'good', 'is', 'not', 'this', 'very'], dtype=object)

The word-count vector can now be used with a suitable model to conduct language processing. Here we will simply use a logit model:

In [4]:
y = ratings

#Logistic regression
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X,y)
print(model.score(X,y))
print(model.predict(X))

0.6666666666666666
[1 0 1 1 0 0]


Which phrases do our model have a difficulty understanding? Why might that be the case?

Now let us take a look at the estimated coefficients:

In [5]:
print(model.coef_)

[[-2.37588867e-01  2.37535611e-01 -3.55988077e-01  4.95675399e-05
  -5.32559462e-05  3.55506184e-01]]


Take a look at the coefficients of each word. Can you see what is wrong with our model? One thing you might notice is that 'is' has a very negative coefficient while 'very' has very a positive coefficient, even though these words do not have such connotations themselves.  

When we start counting combination of words instead of individual words, what we have is *n-gram*. ```CountVectorizer``` allows us to specify the range of words we wish to consider via the option ```ngram_range```:

In [6]:
vectorizer = CountVectorizer(ngram_range=(2,2))
X = vectorizer.fit_transform(corpus)
print(X.toarray())
print(vectorizer.get_feature_names_out())

[[0 1 0 0 0 0 0 1 0]
 [1 0 0 0 0 0 0 1 0]
 [0 0 0 0 1 0 0 1 1]
 [0 0 0 1 0 0 1 1 0]
 [0 0 0 1 0 1 0 1 0]
 [1 0 1 0 0 0 0 1 0]]
['is bad' 'is good' 'is is' 'is not' 'is very' 'not bad' 'not good'
 'this is' 'very good']


Now let us try running the logistic regression again:

In [7]:
model = LogisticRegression()
model.fit(X,y)
print(model.score(X,y))
print(model.coef_)

1.0
[[-6.65676508e-01  3.78679647e-01 -2.99770295e-01 -3.25313983e-02
   3.19546774e-01  3.84908615e-01 -4.17440013e-01  1.85142328e-05
   3.19546774e-01]]


Much better!

### B. IMDB Movie Review

Now let us try something real. We will analyse a sample of <a href="https://www.imdb.com/">IMDB</a> movie reviews, trying to predict the rating a user gives based on his written review. For speed reasons we will be using a subsample, but the original text data can be found <a href="http://ai.stanford.edu/~amaas/data/sentiment/">here</a>.

First let us import the data:

In [1]:
import pandas as pd
imdb_train = pd.read_csv("../Data/imdb_train.csv",
                         names=['label','text'])
imdb_test = pd.read_csv("../Data/imdb_test.csv",
                         names=['label','text'])

How many samples do we have?

In [9]:
print(imdb_train.shape)
print(imdb_test.shape)

(1000, 2)
(1000, 2)


What is inside each sample?

In [10]:
imdb_train.head(5)

,label,text
0,0,This was an absolutely terrible movie. Don't b...
1,0,"I have been known to fall asleep during films,..."
2,0,Mann photographs the Alberta Rocky Mountains i...
3,1,This is the kind of film for a snowy Sunday af...
4,1,"As others have mentioned, all the women that g..."


<!--Words are encoded by their frequency-of-apperance ranking in the data. This allows us to easily delete words that either
- appear frequently but add little to the meaning of the text (e.g. articles, conjunctions and prepositions), or
- appear too infrequently to be of use.-->

We will now repeat what we have done previously:

In [11]:
y_train = imdb_train['label']
y_test = imdb_test['label']

vectorizer = CountVectorizer()
x_train = vectorizer.fit_transform(imdb_train['text'])
x_test = vectorizer.transform(imdb_test['text'])

How well does our model do?

In [12]:
model = LogisticRegression(max_iter=1000)
model.fit(x_train,y_train)
print(model.score(x_train,y_train))
print(model.score(x_test,y_test))

1.0
0.786


### C. Lemmatization

Consider the following corpus of text, modified from the original one:

In [13]:
# Text data
corpus2 = [
    "Apple is good.",
    "Apple was bad.",
    "Apples are good.",
    "Apples were not good.",
    "Apple is not bad.",
    "Apples were...are bad."
]

Having plurals complicates our analysis: `CountVectorizer` will treat 'Apple' and 'Apples' as two distinct words, unncessarily splitting the samples for apples. Similarly, 'is' and 'are' are both forms of the verb 'to be', so they should be considered as one word. What we need is *lemmatization*, which is the process of grouping together the inflected forms of a word for use in analysis.

We will be using <a href="https://textblob.readthedocs.io/en/dev/index.html">TextBlob</a>, a library for processing textual data. TextBlob in turn relies on <a href="http://www.nltk.org/">NLTK</a> (short for *Natural Language ToolKit*) to do some of the heavy lifting. Since NLTK does not come with all packages installed, we will need to first download the ones we need:

In [14]:
import nltk
nltk.download('punkt') 
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/users/testuser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/users/testuser/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/users/testuser/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/users/testuser/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

The process goes as follows:
1. First convert each string to a `TextBlob` object. 
2. Split each string into sentences with the `.sentences` property if needed.
3. Split each string (or sentence) into words with the `.words` property.
4. Lemmatize each word with the `lemmatize()` method. 

Note that `lemmatize()` expects words to be in lowercase.

In [15]:
# Use TextBlob to lemmatize the corpus
from textblob import TextBlob

tb = [TextBlob(c.lower()) for c in corpus2]
sentences = [t.words for t in tb]
data = [s.lemmatize() for s in sentences]
data

[WordList(['apple', 'is', 'good']),
 WordList(['apple', 'wa', 'bad']),
 WordList(['apple', 'are', 'good']),
 WordList(['apple', 'were', 'not', 'good']),
 WordList(['apple', 'is', 'not', 'bad']),
 WordList(['apple', 'were', 'are', 'bad'])]

The code above successfully grouped 'apples' with 'apple', but it failed to group 'is' and 'are'. The second sample gives us some hint as to what went wrong---'was' was somehow converted to 'wa'. What happened was that `lemmatize()` by default treats all words as nouns. To ensure proper conversion, we will need to provide it with each word's part of speech (POS).

First, we generate part-of-speech tags by using the `.tags` property of the `TextBlob` object:


In [16]:
# Extract Penn Treebank POS
tags = [t.tags for t in tb]
tags

[[('apple', 'NN'), ('is', 'VBZ'), ('good', 'JJ')],
 [('apple', 'NN'), ('was', 'VBD'), ('bad', 'JJ')],
 [('apples', 'NNS'), ('are', 'VBP'), ('good', 'JJ')],
 [('apples', 'NNS'), ('were', 'VBD'), ('not', 'RB'), ('good', 'JJ')],
 [('apple', 'NN'), ('is', 'VBZ'), ('not', 'RB'), ('bad', 'JJ')],
 [('apples', 'NNS'), ('were', 'VBD'), ('are', 'VBP'), ('bad', 'JJ')]]

We can then providing `lemmatize()` with part-of-speech tags. Unfortunately it is not as simple as passing the POS tags from above. The reason is that NLTK generates tags base on the <a href="https://catalog.ldc.upenn.edu/LDC99T42">Penn Treebank</a> corpus, which uses different <a href="https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html">POS</a> tags than the <a href="https://wordnet.princeton.edu/documentation/wndb5wn">Wordnet</a> corpus that `lemmatize()` is based on. 

We therefore need to map the two POS systems before lemmatization:

In [17]:
# Function to map Penn Treebank POS to Wordnet POS
def pos_conv(pos):
    tag_dict = {"J": 'a', 
                "N": 'n', 
                "V": 'v', 
                "R": 'r'}    
    return tag_dict.get(pos[0], 'n')

# Convert Penn Treebank POS to Wordnet POS
wordnet_tags = [[[w, pos_conv(pos)] for w, pos in t] for t in tags]

# Lemmatize with POS
data = [[w.lemmatize(t) for w,t in s] for s in wordnet_tags]
data

[['apple', 'be', 'good'],
 ['apple', 'be', 'bad'],
 ['apple', 'be', 'good'],
 ['apple', 'be', 'not', 'good'],
 ['apple', 'be', 'not', 'bad'],
 ['apple', 'be', 'be', 'bad']]

TextBlob and NLTK have many other useful features such as spelling correction and translation that you can explore on your own. One particularly useful feature is pre-trained sentiment analysis:

In [18]:
# Sentiment analysis with TextBlob
sentiment =  [t.sentiment for t in tb]
sentiment

[Sentiment(polarity=0.7, subjectivity=0.6000000000000001),
 Sentiment(polarity=-0.6999999999999998, subjectivity=0.6666666666666666),
 Sentiment(polarity=0.7, subjectivity=0.6000000000000001),
 Sentiment(polarity=-0.35, subjectivity=0.6000000000000001),
 Sentiment(polarity=0.3499999999999999, subjectivity=0.6666666666666666),
 Sentiment(polarity=-0.6999999999999998, subjectivity=0.6666666666666666)]

### D. Chinese Text

One major issue with Chinese text is that there is no space between words. Unsurprisingly then, this is a major focus for Chinese natural language processing research.

They are multiple libraries for Chinese NLP. Here we will try out `jieba` and `pkuseg`.

In [19]:
text = '我愛吃北京餃子。'

# jieba default
import jieba
seg_list = jieba.cut(text) 
print([w for w in seg_list])

# jieba cut all mode
import jieba
seg_list = jieba.cut(text, cut_all=True) 
print([w for w in seg_list])

# jieba + paddle
import paddle
paddle.enable_static()
jieba.enable_paddle()
seg_list = jieba.cut(text,use_paddle=True)
print([w for w in seg_list])

# pkuseg
import spacy_pkuseg as pkuseg
seg = pkuseg.pkuseg() 
seg_list = seg.cut(text)
print(seg_list)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.657 seconds.
Prefix dict has been built successfully.


['我愛吃', '北京', '餃子', '。']
['我', '愛', '吃', '北京', '餃', '子', '。']


Import error, cannot find paddle.fluid and jieba.lac_small.predict module. Now, back to jieba basic cut......
[2024-10-14 12:43:28,023] [   DEBUG] _compat.py:50 - Import error, cannot find paddle.fluid and jieba.lac_small.predict module. Now, back to jieba basic cut......


['我愛吃', '北京', '餃子', '。']
['我', '愛', '吃', '北京', '餃子', '。']


Things are much easier once we have the individual words. For example, we could immediately use ngram on the text.

We can also fetch POS:

In [20]:
# jieba
import jieba.posseg as pseg
words = pseg.cut(text)
print([(w,f) for w,f in words])

# jieba + paddle
import paddle
paddle.enable_static()
jieba.enable_paddle()
words = pseg.cut(text,use_paddle=True)
print([(w,f) for w,f in words])

# pkuseg
seg = pkuseg.pkuseg(postag=True)
seg_list = seg.cut(text)
print(seg_list)

Import error, cannot find paddle.fluid and jieba.lac_small.predict module. Now, back to jieba basic cut......
[2024-10-14 12:43:29,189] [   DEBUG] _compat.py:50 - Import error, cannot find paddle.fluid and jieba.lac_small.predict module. Now, back to jieba basic cut......


[('我', 'r'), ('愛', 'v'), ('吃', 'v'), ('北京', 'ns'), ('餃子', 'n'), ('。', 'x')]
[('我', 'r'), ('愛', 'v'), ('吃', 'v'), ('北京', 'ns'), ('餃子', 'n'), ('。', 'x')]
[('我', 'r'), ('愛', 'v'), ('吃', 'v'), ('北京', 'ns'), ('餃子', 'n'), ('。', 'w')]


POS tags for `pkugseg`:
https://github.com/lancopku/pkuseg-python/blob/master/tags.txt

For `jieba`:
https://github.com/fxsjy/jieba

### E. Neural Network

Below is a simple LSTM neural network model that runs sentiment analysis on the IMDB data:

In [21]:
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
from keras.utils import pad_sequences
from sklearn.utils import resample

max_features = 20000
maxlen = 80  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train,y_train,x_test,y_test = resample(x_train,y_train,x_test,y_test,
                                         n_samples=1000)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2))
model.add(Dense(128))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=15,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

2024-10-14 12:43:30.665821: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-14 12:43:30.665943: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-14 12:43:30.699404: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-14 12:43:30.771625: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-14 12:43:32.833444: W tensorflow/compiler/tf2

Loading data...
1000 train sequences
1000 test sequences
Pad sequences (samples x time)
x_train shape: (1000, 80)
x_test shape: (1000, 80)
Build model...


2024-10-14 12:43:37.664373: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22361 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:c1:00.0, compute capability: 8.6
2024-10-14 12:43:38.000351: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: Permission denied


Train...
Epoch 1/15


2024-10-14 12:43:40.463256: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-10-14 12:43:40.927002: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f8e7a0e05b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-10-14 12:43:40.927037: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-10-14 12:43:40.938480: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1728881021.039628 3529329 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


32/32 [==============================] - 6s 102ms/step - loss: 0.6838 - accuracy: 0.5520 - val_loss: 0.6425 - val_accuracy: 0.6890
Epoch 2/15
32/32 [==============================] - 2s 73ms/step - loss: 0.4528 - accuracy: 0.8270 - val_loss: 0.5539 - val_accuracy: 0.7210
Epoch 3/15
32/32 [==============================] - 2s 53ms/step - loss: 0.1439 - accuracy: 0.9620 - val_loss: 0.7331 - val_accuracy: 0.7390
Epoch 4/15
32/32 [==============================] - 1s 34ms/step - loss: 0.0307 - accuracy: 0.9930 - val_loss: 1.0014 - val_accuracy: 0.7350
Epoch 5/15
32/32 [==============================] - 2s 51ms/step - loss: 0.0107 - accuracy: 0.9990 - val_loss: 0.9336 - val_accuracy: 0.7290
Epoch 6/15
32/32 [==============================] - 1s 29ms/step - loss: 0.0010 - accuracy: 1.0000 - val_loss: 1.2709 - val_accuracy: 0.7420
Epoch 7/15
32/32 [==============================] - 1s 26ms/step - loss: 2.3477e-04 - accuracy: 1.0000 - val_loss: 1.1420 - val_accuracy: 0.7470
Epoch 8/15
32/32 [=

You should notice that training a neural network is several orders of magnitude slower than a n-gram model. Furthermore, the neural network model above is not more accurate than our simple n-gram model. One reason is that with so many parameters, neural network models need more than a thousand sample to achieve good results if you are training one from scratch. You can try running the same script with more data on a computer with GPU and see whether you get better results.



### F. Large Language Models

A much better way to incorporate a neural network models is to use a *pre-trained* language model, 
which has been trained to understand language based on an enormous amount of text data.
The main reason why the models we have tried so far do not work well is that they have to learn
English from scratch based on the relatively small number of samples we provide. 
The use of a pre-trained model circumvent this issue. 

We will go into the details of such pre-trained models in a later lecture.
Here we will simply have a demo. As running these models are very computationally intensive,
you should run the following code with access to GPU, otherwise it is going to be very slow.

In [7]:
# Run with GPU! This is very slow on CPU.

# Data
x_test = imdb_test['text'].tolist()
y_test = imdb_test['label'].tolist()

# Use a pre-trained text classifier through Hugging Face transformer library
from transformers import pipeline
classifier = pipeline('text-classification', device=0) # device=0 means use (first) GPU
tokenizer_kwargs = {'truncation':True,'max_length':512} # Truncate to 512 tokens
results = classifier(x_test,**tokenizer_kwargs) # Returns a text label
y_predicted = [ 1 if x['label']=='POSITIVE' else 0 for x in results] # Convert to 1 or 0

# Measure accuracy
from sklearn.metrics import accuracy_score
print("Accuracy:",accuracy_score(y_test,y_predicted))

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


4.35


0.882

### G. Accessing Language Models through API

Instead of running your own copy pf a pre-trained language model,
you can defer the task to a remotely-hosted model. The most well-known
remotely-hosted models are OpenAI's GPTs and Anthropic's Claude.
Here we will use Meta's Llama 3.1, hosted on CUHK Department of Economics'
servers. 

Note that while this is the most powerful and convenient method of conducting 
text classification, it is also the slowest&mdash;you pay a price for being able 
to give the model instuctions in plain language.

In [ ]:
# Text classification through API

# Need to supply your own API key
api_key = 'your-key-here'

from openai import OpenAI
import numpy as np

# Data - using only 30 samples for speed reasons
x_test = imdb_test['text'].head(30).tolist()
y_test = imdb_test['label'].head(30).tolist()

# OpenAI API
client = OpenAI(
    base_url = 'https://scrp-chat.econ.cuhk.edu.hk/api',
    api_key=api_key,
)

# Function for running the inference
def f(x):
    response = client.chat.completions.create(
      model="llama3.1:8b-instruct-q5_K_M",
      messages=[
        {"role": "system", 
         "content": """Please classify if the given text's sentiment is positive or negative.
                       If it is positive, return 1. Otherwise return 0. 
                       Show only the finally answer, do not show your reasoning.
                    """},        
        {"role": "user", "content": x}
      ],
      temperature=0  
    )
    return int(response.choices[0].message.content)

# Loop through data
y_predicted = [f(x) for x in x_test]

# Measure accuracy
from sklearn.metrics import accuracy_score
print("Accuracy:",accuracy_score(y_test,y_predicted))

### Further Readings
- <a href="https://github.com/dipanjanS/text-analytics-with-python">Text Analytics with Python</a> (or the <a href="https://towardsdatascience.com/a-practitioners-guide-to-natural-language-processing-part-i-processing-understanding-text-9f4abfd13e72">free tutorial</a> by the same author on Towards Data Science.)